In [1]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

ERROR:root:Line magic function `%install_ext` not found.


In [2]:
# read in known applicant files, dedupe and try to merge with applicants file
from collections import Counter
import difflib
import uuid
import itertools
import pandas as pd
import numpy as np
import string
import funcy
import re
import os

from data_cleaning_functions import (trans_remov_punc, standardize_whitespace, remove_punc, remove_suffix_from_last_name,
                                     clean_names, has_award, has_suffix, get_suffix, replace_last_name, 
                                     is_year_range, str_sim, clean_med_school, clean_std_college_name, long_form_date, 
                                    correct_mispellings, is_foreign_med_school)

from dev import (
    APP_DATA_DIR, SUM_STAT_DIR, ATT_DATA_DIR, CARD_DATA_DIR, CORRECTIONS_DIR, AWARDS_KEYWORDS, NAME_COLS, RAW_NAME_COLS, 
    RAW_CARD_ID, RAW_INDEX_IDS, PERSON_APPLICATION_ID, PERSON_ID, NIH_ID, FEMALE_FIRST_NAMES, FEMALE_MIDDLE_NAMES, 
    PICKLE_DIR)
from merging_functions import *

OUTPUT_CSV = False 

PERSONAL_INFO = [
    'clean_first_name', 'clean_last_name', 'clean_middle_name',
    'date_of_birth', 'medical_school', 'clean_college_trans']


time: 239 ms


In [3]:
# id column that links back to raw applicant data file
RAW_CARD_ID = 'raw_uuid'

# column where the raw id information is stored
RAW_INDEX_IDS = 'raw_card_ids'

# try to get one id per unique applicant in the dataset
PERSON_ID = 'person_uuid'
# id per deduped application-person - if someone applied multiple times, they will have multiple ids
PERSON_APPLICATION_ID = 'person_app_uuid' 
NIH_ID = 'dno'

APPLICANT_SUFFIX = '_ap'
ATTENDEE_SUFFIX = '_at'

%load_ext autoreload
%autoreload 2

%reload_ext autoreload

time: 38.1 ms


In [4]:
apps_filename = 'index_cards_deduped_fuzzy.csv'
# apps_filename = 'person_application_date_wide.csv'

NIH_filename = 'unique_attendees.csv'
# NIH_filename = 'NIH_attendee_deduped_raw.csv'

time: 11 ms


In [5]:
NAME_COLS = ['clean_middle_name', 'clean_last_name', 'clean_first_name']

MED_TRAINING_COLS = ['res_dates', 'intern_dates', 'residency_hospital', 'internship_hospital', 'medical_school', 'residency']


time: 11 ms


In [6]:
# import NIH raw data set
NIH_raw = pd.read_csv(os.path.join(ATT_DATA_DIR, NIH_filename)).drop_duplicates('dno')
sorted(NIH_raw.columns)

['Unnamed: 0',
 'citizenship',
 'clean_first_initial',
 'clean_first_name',
 'clean_last_name',
 'clean_medical_school',
 'clean_middle_initial',
 'clean_middle_name',
 'clean_suffix',
 'count_missing',
 'data_source',
 'dno',
 'dob',
 'dup_flag',
 'duplicate_dno',
 'eod_year',
 'eod_year_diff',
 'generation',
 'institute',
 'intern_dte',
 'intern_hos',
 'lab_brch',
 'med_school',
 'medical_school',
 'program',
 'res_dtes',
 'res_hosp',
 'residency',
 'source',
 'ssn',
 'supervisor',
 'unknown',
 'year_grad']

time: 46.4 ms


In [7]:
# drop from the data set all people with eod years > 1980
# NIH = NIH_raw.loc[NIH_raw.eod_year<1980, :] 
NIH = NIH_raw
del NIH['medical_school']

time: 12.6 ms


In [8]:
# import cleaned, deduped applicant data set in wide form (multiple app dates as columns)
apps = pd.read_csv(os.path.join(APP_DATA_DIR, apps_filename))


time: 64.1 ms


In [9]:
apps.loc[apps.clean_last_name=='BURNBAUM', NAME_COLS+['medical_school', 'application_year']]

,clean_middle_name,clean_last_name,clean_first_name,medical_school,application_year
2638,DAVID,BURNBAUM,MITCHELL,UNIVERSITY OF VIRGINIA SCHOOL OF MEDICINE,1972.0


time: 24.9 ms


In [10]:
apps.loc[:, 'res_dates'] = apps['residency_year(s)'].apply(long_form_date)

apps.loc[:, 'intern_dates'] = apps['internship_year(s)'].apply(long_form_date)

"Open"
Str. Medicine
Medicine
time: 68.8 ms


In [11]:
NIH = NIH.rename(columns={'res_dtes':'res_dates', 'intern_dte':'intern_dates', 'intern_hos': 'internship_hospital', 
                         'res_hosp':'residency_hospital', 'clean_medical_school': 'medical_school'})
# sorted(NIH.columns)

time: 13.9 ms


In [12]:
apps2 = apps.rename(
    columns={'residency_type': 'residency', 'internship_hospital_1': 'internship_hospital'})

string_med_cols = ['medical_school', 'residency_hospital', 'internship_hospital', 'residency']

# apply string cleaning function to each of the string medical info columns
apps2.loc[:, string_med_cols] = apps2[string_med_cols].applymap(clean_names)

time: 169 ms


In [13]:
to_remove = ['TERRECE', 'FRED', 'LAURENCE',
             'CUONO', 'DEFRENZE', 'JEFFERY', 'FINKLEMAN', 'SHERRAD', 'ANSCHNETZ', 'MARC', 'JENSON', 'KASTI', 
            'ADELBERT', 'RITCHARD', 'MANSFORD', 'DEFRENZO', 'DROBIN', 'HAMES', 'KREUZ', 'JERROLD', 'MANEUSI',
            'UNGARO']
to_replace = ['TERRENCE', 'FREDERICK', 'LAWRENCE',
              'CUOMO', 'DEFRONZO', 'JEFFREY', 'FINKELMAN', 'SHERRARD', 'ANSCHUETZ', 'MARCUS', 'JENSEN', 'KASTL',
              'ALBERT', 'RITCHARD', 'MANIFORD', 'DEFRONZO', 'DROBIS', 'JAMES', 'KRUEZ', 'JERROD', 'MANCUSI',
              'UNGARO']

correct_name_mispellings_fnc = funcy.rpartial(correct_mispellings, to_remove, to_replace)

apps2.loc[:, 'clean_last_name'] = apps2.clean_last_name.apply(correct_name_mispellings_fnc)
apps2.loc[:, 'clean_first_name'] = apps2.clean_first_name.apply(correct_name_mispellings_fnc)

time: 71.9 ms


In [14]:
apps2.loc[apps2.clean_last_name=='BRESLOW']

,person_uuid,address,age,application_date,application_year,application_year_1,application_year_2,application_year_3,associate_program_entered,bob,...,state,teaching,undergrad_year_grad,undergraduate_school,withdrawal,year_accepted,zip_code,to_drop,res_dates,intern_dates
1621,1619,612 Avenue K,NaN,1968-03-21,1968.0,NaN,NaN,NaN,NaN,0.0,...,New York,1.0,NaN,NaN,-9.0,1970.0,NaN,False,1969-1970,1968-1969


time: 44.1 ms


In [15]:
# function to go in and correct some of the name mispellings in both data sets
# MUTATING FUNCTION
def change_names(df, selection_type, selection_value, to_change_type, to_change_values):
    for t, v in zip(to_change_type, to_change_values):
        print t, v
        sel = df.loc[df[selection_type]==selection_value, t]
        print sel.shape[0]
        if sel.shape[0] > 1:
            print "WARNING"
        df.loc[df[selection_type]==selection_value, t] = v

time: 15.3 ms


In [16]:

apps2.loc[(apps2.clean_last_name=='LIBOW') & (apps2.clean_middle_name=='S'), 'clean_first_name'] = 'LESLIE'

change_names(
    apps2, 'clean_last_name', 'CHESEBRO', ['clean_first_name', 'clean_middle_name'], ['BRUCE', 'WILCOX'])
change_names(
    apps2, 'clean_last_name', 'GALANTER', ['clean_first_name', 'clean_middle_name'], ['MARC', 'I'])
change_names(
    apps2, 'clean_last_name', 'BEAN', ['clean_first_name', 'clean_middle_name', 'medical_school'], ['SIDNEY', 'CHARLES', 'WAKE_FOREST'])
change_names(
    apps2, 'clean_last_name', 'BOYD', ['clean_first_name', 'clean_middle_name'], ['MICHAEL', 'RAY'])
change_names(
    apps2, 'clean_last_name', 'CHAPMAN', 
    ['clean_first_name', 'clean_middle_name', 'medical_school'], ['STANLEY', 'WILLETS', 'ROCHESTER'])

clean_first_name BRUCE
1
clean_middle_name WILCOX
1
clean_first_name MARC
1
clean_middle_name I
1
clean_first_name SIDNEY
1
clean_middle_name CHARLES
1
medical_school WAKE_FOREST
1
clean_first_name MICHAEL
1
clean_middle_name RAY
1
clean_first_name STANLEY
1
clean_middle_name WILLETS
1
medical_school ROCHESTER
1
time: 38.9 ms


In [17]:
apps2.loc[apps2.clean_last_name=='JAN', ]

,person_uuid,address,age,application_date,application_year,application_year_1,application_year_2,application_year_3,associate_program_entered,bob,...,state,teaching,undergrad_year_grad,undergraduate_school,withdrawal,year_accepted,zip_code,to_drop,res_dates,intern_dates
3452,3452,1248-42nd Avenue,NaN,1971-04-03,1971.0,NaN,NaN,NaN,NaN,0.0,...,California,1.0,NaN,NaN,-9.0,NaN,95822.0,False,NaN,1971


time: 38.9 ms


In [18]:
change_names(apps2, 'clean_last_name', 'DANFORTH', ['clean_first_name'], ['DAVID'])
change_names(apps2, 'clean_last_name', 'HUNT', ['clean_first_name', 'clean_middle_name'], ['ROBERT', 'D'])
change_names(apps2, 'clean_last_name', 'KARK', ['clean_first_name', 'clean_middle_name'], ['ROBERT', 'ADRIAN'])
change_names(apps2, 'clean_last_name', 'KEBABIAN', ['clean_first_name', 'clean_middle_name'], ['JOHN', 'WILLIS'])
change_names(apps2, 'clean_last_name', 'KNOPF', ['clean_first_name', 'clean_middle_name'], ['HARRY', 'LOUIS'])
change_names(apps2, 'clean_last_name', 'KROLIKOWSKI', ['clean_first_name', 'clean_middle_name'], ['FRANCIS', 'JOHN'])
change_names(apps2, 'clean_last_name', 'KASTL', ['clean_first_name', 'clean_middle_name'], ['DAVID', 'GENE'])
change_names(apps2, 'clean_last_name', 'KLAVEMAN', ['clean_last_name'], ['KLAEVEMAN'])
change_names(apps2, 'clean_last_name', 'MATHEW', ['clean_last_name'], ['MATTHEW'])

apps2.loc[apps2.clean_last_name=='CHESEBRO', ['clean_first_name']] = 'BRUCE'
apps2.loc[apps2.clean_last_name=='CHESEBRO', ['clean_middle_name']] = 'WILCOX'
apps2.loc[(apps2.clean_last_name=='HEALY') & (apps2.medical_school=='USC KECK SCHOOL OF MEDICINE'), ['clean_first_name']] = 'MARK'
apps2.loc[(apps2.clean_last_name=='HEALY') & (apps2.medical_school=='USC KECK SCHOOL OF MEDICINE'), ['clean_middle_name']] = 'H'

apps2.loc[apps2.clean_last_name=='LENN', 'clean_first_name'] = 'NICHOLAS'
apps2.loc[apps2.clean_last_name=='BRESLOW', 'clean_first_name'] = 'JAN'
apps2.loc[apps2.clean_last_name=='BRESLOW', 'clean_middle_name'] = 'LESLIE'

apps2.loc[(apps2.clean_last_name=='NADLER') & (pd.isnull(apps2.clean_first_name)), 'clean_first_name'] = 'LEE'
apps2.loc[(apps2.clean_last_name=='NADLER') & (apps2.clean_first_name=='LEE'), 'clean_middle_name'] = 'MARSHALL'
apps2.loc[(pd.isnull(apps2.clean_first_name)) & (apps2.clean_last_name=='ROSEN'), 'clean_first_name'] = 'HENRY'

apps2.loc[(apps2.clean_last_name=='NEELON'), 'clean_first_name'] = 'FRANCIS'
apps2.loc[(apps2.clean_last_name=='NEELON') , 'clean_middle_name'] = 'ALBERT'

apps2.loc[(apps2.clean_last_name=='NICHOLAS') , 'clean_first_name'] = 'JOHN'

clean_first_name DAVID
1
clean_first_name ROBERT
0
clean_middle_name D
0
clean_first_name ROBERT
1
clean_middle_name ADRIAN
1
clean_first_name JOHN
1
clean_middle_name WILLIS
1
clean_first_name HARRY
1
clean_middle_name LOUIS
1
clean_first_name FRANCIS
1
clean_middle_name JOHN
1
clean_first_name DAVID
1
clean_middle_name GENE
1
clean_last_name KLAEVEMAN
0
clean_last_name MATTHEW
0
time: 78.5 ms


In [19]:
# there are a bunch of duplicates in apps, where application year is the same, but first name is missing
name_dups = apps2.loc[
    apps2.duplicated(
        ['clean_last_name', 'medical_school', 'application_year'], keep=False), NAME_COLS+[PERSON_ID, 'medical_school', 'application_year']]

time: 15.5 ms


In [20]:
to_delete_ids = name_dups.loc[(pd.isnull(name_dups.clean_middle_name)) & (pd.isnull(name_dups.clean_first_name))
                             & pd.isnull(name_dups.medical_school), PERSON_ID]

time: 14.2 ms


In [21]:
apps4 = apps2.loc[~apps2[PERSON_ID].isin(to_delete_ids), :].sort_values(
    NAME_COLS+['medical_school'], ascending=False).drop_duplicates(NAME_COLS+['medical_school'])

time: 31.8 ms


In [22]:
print apps4.shape
print apps2.shape

(4159, 85)
(4168, 85)
time: 10.8 ms


In [23]:
NIH.loc[(NIH.clean_last_name=='ALEXANDER') & (NIH.clean_first_name=='JOHN'), 'clean_middle_name'] = 'CHARLES'

time: 14.1 ms


In [24]:
NIH.loc[(NIH.clean_last_name=='LEBOWITZ') & (NIH.clean_first_name=='EDWARD ARTHUR'),'clean_middle_name'] = 'ARTHUR'
NIH.loc[(NIH.clean_last_name=='LEBOWITZ') & (NIH.clean_first_name=='EDWARD ARTHUR'),'clean_first_name'] = 'EDWARD'
NIH.loc[(
        NIH.clean_last_name=='LEBOWITZ') & (
            NIH.clean_first_name=='EDWARD'), 'medical_school'] = 'ALBERT EINSTEIN COLLEGE OF MEDICINE OF YESHIVA'

time: 19.7 ms


In [25]:
NIH = NIH.loc[NIH.clean_first_name!='GERALDINE']

time: 14 ms


In [26]:
# read in manual matches
man = pd.read_excel(os.path.join(CORRECTIONS_DIR, 'manual_dno_matches.xlsx'), index=False).rename(columns={'medical_school': 'dno_medical_school'})
man['medical_school'] = man.dno_medical_school.apply(clean_med_school)

time: 115 ms


In [27]:
t = pd.merge(left=apps4, right=man, on=['clean_last_name', 'clean_middle_name'], how='inner', suffixes=['_x', '_y'])
t.shape

t['sim'] = t[['medical_school_x', 'medical_school_y']].apply(get_name_str_sim, axis=1)

t['clean_first_name'] = t['clean_first_name_y']

t_1 = t.sort_values(['dno', 'sim'], ascending=False).drop_duplicates(['dno'], keep='first')

# t[NAME_COLS+['dno', 'medical_school_x', 'medical_school_y', 'sim']]
t_1.loc[t_1.duplicated('dno', keep=False), NAME_COLS+['dno', 'medical_school_x', 'medical_school_y', 'sim']]

,clean_middle_name,clean_last_name,clean_first_name,dno,medical_school_x,medical_school_y,sim


time: 46.5 ms


In [28]:
apps4.loc[apps4.clean_last_name=='BRESLOW', NAME_COLS+['application_year']]

,clean_middle_name,clean_last_name,clean_first_name,application_year
1621,LESLIE,BRESLOW,JAN,1968.0


time: 26 ms


In [29]:
um = man[~man.dno.isin(t_1.dno)]

um.shape

(0, 8)

time: 15.1 ms


In [30]:
um

,dno,clean_first_name,clean_middle_name,clean_last_name,application_year_min,application_year_max,dno_medical_school,medical_school


time: 16.9 ms


In [31]:
# t2 = pd.merge(left=apps4, right=um, on=['clean_last_name', 'clean_first_name'], how='inner', suffixes=['_x', '_y'])
# print t2.shape
# print um.shape

# t2['sim'] = t2[['medical_school_x', 'medical_school_y']].apply(get_name_str_sim, axis=1)

# t2['clean_middle_name'] = t2['clean_middle_name_y']

# t2_1 = t2.sort_values(['dno', 'sim'], ascending=False).drop_duplicates(['dno'], keep='first')
# t2_1.shape

time: 11.1 ms


In [32]:
t3 = t_1[NAME_COLS+['dno', PERSON_ID]]
# t3 = pd.concat([t2_1[NAME_COLS+['dno', PERSON_ID]], t_1[NAME_COLS+['dno', PERSON_ID]]], axis=0)

time: 12.5 ms


In [33]:
man_dno = pd.merge(left=apps4, right=t3, on=PERSON_ID, how='left', suffixes=['_x', '_y'])
print man_dno.shape
print t3.shape
print man.shape

(4159, 89)
(103, 5)
(104, 8)
time: 27.6 ms


In [34]:
mask = ~pd.isnull(man_dno.clean_last_name_y)
man_dno = man_dno.rename(columns={'clean_last_name_x': 'clean_last_name', 'clean_first_name_x': 'clean_first_name', 
                       'clean_middle_name_x': 'clean_middle_name'})
mask = ~pd.isnull(man_dno.clean_last_name_y)
man_dno.loc[mask, 'clean_last_name'] = man_dno.loc[mask, 'clean_last_name_y']
mask = ~pd.isnull(man_dno.clean_first_name_y)
man_dno.loc[mask, 'clean_first_name'] = man_dno.loc[mask, 'clean_first_name_y']
mask = ~pd.isnull(man_dno.clean_middle_name_y)
man_dno.loc[mask, 'clean_middle_name'] = man_dno.loc[mask, 'clean_middle_name_y']

time: 32.7 ms


In [35]:
apps5 = man_dno.loc[pd.isnull(man_dno['dno']), :].drop(['dno', 'clean_middle_name_y', 'clean_first_name_y', 'clean_last_name_y'], axis=1) 

time: 19.1 ms


In [36]:
# remove females from data set
female_mask = (NIH.clean_first_name.isin(FEMALE_FIRST_NAMES))  
NIH.loc[female_mask, 'is_female'] = 1


female_mask = (apps5.clean_first_name.isin(FEMALE_FIRST_NAMES))  
apps5.loc[female_mask, 'is_female'] = 1

time: 38.9 ms


In [37]:
def get_first_letter(str_var):
    if pd.isnull(str_var) or str_var=='':
        return np.nan
    return str_var[0]

time: 11.3 ms


In [38]:
NIH['clean_first_initial'] = NIH.clean_first_name.apply(get_first_letter)
NIH['clean_middle_initial'] = NIH.clean_middle_name.apply(get_first_letter)

time: 17.4 ms


In [39]:
print NIH.shape
print  apps5.shape

(4012, 33)
(4056, 85)
time: 10.9 ms


In [40]:
# After cleaning apps2 to match cleaning in Clean NIH Applicant notebook, we try to start merging
sims_cols = ['medical_school_sim', 'clean_middle_name_sim', 'clean_first_name_sim']

time: 10.5 ms


In [41]:
NIH['fuzzy_merge_col'] = NIH[
    ['clean_first_name', 'clean_middle_name', 'clean_last_name']].apply(create_str_merge, axis=1)
apps5['fuzzy_merge_col'] = apps5[
    ['clean_first_name', 'clean_middle_name', 'clean_last_name']].apply(create_str_merge, axis=1)
match1 = df_get_closest_matches(apps5, NIH, 'fuzzy_merge_col', suffixes=['_x', '_y']) 
print match1.shape

(4012, 118)
time: 5min 37s


In [42]:
# add last name counter to each
app_counter = Counter(apps4.clean_last_name.values)
NIH_counter = Counter(NIH.clean_last_name.values)
match1['last_name_counts_x'] = match1.clean_last_name_x.apply(lambda x: app_counter[x])
match1['last_name_counts_y'] = match1.clean_last_name_y.apply(lambda x: NIH_counter[x])


time: 27.8 ms


In [43]:
def check_match(row):
    # address and application year match
    app_eod_year_diff = abs(row['application_year'] - row['eod_year'])
    if row['clean_last_name_sim'] < 60 or app_eod_year_diff > 8:
        return 0
    
    # the first and middle name seem to be mixed up in index card data set
    mixed_sim1 =  get_name_str_sim(row[['clean_middle_name_x', 'clean_first_name_y']])
    mixed_sim2 =  get_name_str_sim(row[['clean_middle_name_y', 'clean_first_name_x']])
    
    mix_sim = max(mixed_sim1, mixed_sim2)
    if pd.isnull(mix_sim):
        mix_sim = 0
    if (mix_sim > 90) and row['medical_school_sim'] > 80:
        return 1
    if row['last_name_counts_x'] < 2 and row['last_name_counts_y'] < 2:
        return 1
    if (mix_sim > 90) and pd.isnull(row['medical_school_sim']) and (app_eod_year_diff < 5):
        return 1
    if not pd.isnull(row['clean_first_name_sim']) and row['clean_first_name_sim'] < 50:
        return 0
    # if matching application year and med schools match
    if (app_eod_year_diff < 5) and row['medical_school_sim'] > 80:
        return 1
    # first and middle names match or first
    if (app_eod_year_diff < 5) and row['clean_first_name_sim'] > 80:
        return 1
    return 0


feature_dict = {
    'clean_first_name': get_name_str_sim,
    'clean_middle_name': get_name_str_sim,
    'clean_last_name': get_name_str_sim,
    'medical_school': get_name_str_sim,
}

match2 = add_similarity_features(match1, feature_dict, check_match, suffixes=['_x', '_y'])

time: 2.85 s


In [44]:
# match2.loc[(match2.is_match==2), sims_cols+['is_match', 'clean_first_name_x', 'clean_first_name_y',
#                         'clean_last_name_x', 'clean_last_name_y','medical_school_x', 'medical_school_y', PERSON_ID, NIH_ID]]

time: 11 ms


In [45]:
# select out people who match and make sure each person id and dno only 1x in data set
match3 = match2[match2.is_match==1].sort_values(['clean_last_name_x']+sims_cols, ascending=False).dropna(
    axis=0, subset=[RAW_CARD_ID])
print match3.shape

(2671, 125)
time: 28.1 ms


In [46]:
# match2[match2.clean_last_name_x=='ADLER']
NIH[NIH.clean_last_name=='ADLER']

,Unnamed: 0,citizenship,clean_first_initial,clean_first_name,clean_last_name,medical_school,clean_middle_initial,clean_middle_name,clean_suffix,count_missing,...,res_dates,residency_hospital,residency,source,ssn,supervisor,unknown,year_grad,is_female,fuzzy_merge_col
2926,0,NaN,R,RICHARD,ADLER,VANDERBILT UNIVERSITY SCHOOL OF MEDICINE,D,DAVID,NaN,1.0,...,1970-1971,Johns Hopkins Hospital,Pediatrics,NIH EDUCATION,NaN,Whedon,NaN,1969.0,NaN,RICHARD DAVID ADLER
3027,2173,NaN,R,RICHARD,ADLER,VANDERBILT UNIVERSITY SCHOOL OF MEDICINE,NaN,NaN,NaN,1.0,...,1960-1961,Johns Hopkins Hospital,Pediatrics,NIH EDUCATION,NaN,NaN,NaN,1959.0,NaN,RICHARD ADLER
3628,1969,NaN,S,STUART,ADLER,JOHNS HOPKINS UNIVERSITY SCHOOL OF MEDICINE,P,PHILLIP,NaN,2.0,...,NaN,NaN,NaN,NIH EDUCATION,NaN,NaN,NaN,1971.0,NaN,STUART PHILLIP ADLER


time: 53.1 ms


In [47]:
def filter_one_match_per_group(df, dedupe_col, sim_cols):
    # to merge cols should be a dict the names of the extra cols to merge in
    # values should be col names to rename
    # sim cols should be name of the columns to use as features
    # sim mask should be mask that accounts as actual mask
    # dedupe col is name of col to dedupe on

    def count_matches(id_list_arr):
        # for each id, make sure matched on 1x in data set
        # should be applied with rolling apply so takes in a dataframe and must return single value
        # unpack already matched ids from string
        current_id1 = id_list_arr[-1]
        other_matches = id_list_arr[:-1]
        is_dup = np.any(other_matches[:] == current_id1)
        if is_dup:
            return True
        return False

    # for each uuid, check for duplicates and choose best match based on sim cols
    # order of the sim cols should be with most important first
    dup_flag = '{}_duplicate'.format(dedupe_col)
    df[dup_flag] = 0
    df.loc[:, dup_flag] = df[
        dedupe_col].expanding(center=False, min_periods=0).apply(func=count_matches)

    df_matches = df[df['is_match'] == 1].sort_values([dedupe_col] + sim_cols, ascending=False)
    return df_matches.drop_duplicates([dedupe_col], keep='first')


time: 17.1 ms


In [48]:
match4 = filter_one_match_per_group(match3, 'raw_uuid', sims_cols)
print match4.shape

(2632, 126)
time: 77.5 ms


In [49]:
match5 = filter_one_match_per_group(match4, NIH_ID, sims_cols)
print match5.shape

(2632, 127)
time: 82 ms


In [50]:
# get nonmatched NIH people and not matched applicants 
nm_apps = get_nonmatched(apps5, id_colname=RAW_CARD_ID, matched_ids=match5[RAW_CARD_ID].dropna().values)
nm_NIH = get_nonmatched(NIH, id_colname=NIH_ID, matched_ids=match5[NIH_ID].dropna().values)

(1420, 86)
(1380, 34)
time: 18.6 ms


In [51]:
# do another round of matching just on last name
nm_match1 = df_get_closest_matches(nm_apps, nm_NIH, 'clean_last_name', suffixes=['_x', '_y']) 
print nm_match1.shape


(2298, 118)
time: 10.6 s


In [52]:
def check_match(row):
    app_eod_year_diff = abs(row['application_year'] - row['eod_year'])
    if app_eod_year_diff > 8:
        return 0
    
    # the first and middle name seem to be mixed up in index card data set
    mixed_sim1 =  get_name_str_sim(row[['clean_middle_name_x', 'clean_first_name_y']])
    mixed_sim2 =  get_name_str_sim(row[['clean_middle_name_y', 'clean_first_name_x']])
    
    max_name_sim = max(row['clean_first_name_sim'], row['medical_school_sim'], row['clean_middle_name_sim'])
    if max_name_sim < 60:
        return 0
    mix_sim = max(mixed_sim1, mixed_sim2)
    if pd.isnull(mix_sim):
        mix_sim = 0
    if (mix_sim > 90) and row['medical_school_sim'] > 60:
        return 1
    if (mix_sim > 90) and pd.isnull(row['medical_school_sim']) and (app_eod_year_diff < 6):
        return 1
    if not pd.isnull(row['clean_first_name_sim']) and row['clean_first_name_sim'] < 40:
        return 0
    # if matching application year and med schools match
    if (app_eod_year_diff < 6) and row['medical_school_sim'] > 80:
        return 1
    if not pd.isnull(row['medical_school_sim']) and row['medical_school_sim'] < 40:
        return 0
    # first and middle names match or first
    if (app_eod_year_diff < 6) and row['clean_first_name_sim'] > 80:
        return 1
    # first and middle names match or first
    if (app_eod_year_diff < 6) and row['clean_middle_name_sim'] > 80:
        return 1
    return 0

time: 23.9 ms


In [53]:
nm_feature_dict = {
    'clean_first_name': get_name_str_sim,
    'clean_middle_name': get_name_str_sim,
    'medical_school': get_name_str_sim,
}

nm_match2 = add_similarity_features(nm_match1, nm_feature_dict, check_match, suffixes=['_x', '_y'])


time: 908 ms


In [54]:
nm_match3 = nm_match2.loc[(nm_match2.is_match==1) & (nm_match2.index!='MORTON'), :].reset_index(
    drop=False).rename(columns={'index': 'clean_last_name'})

time: 15.6 ms


In [55]:
# merge the manual people with dno with the dno data set and append to the matches list
man_dno2 = man_dno.loc[~pd.isnull(man_dno['dno']), :].drop(['clean_first_name_y', 'clean_middle_name_y',
                                                            'clean_last_name_y'], axis=1)
man_dno2.loc[:, 'dno'] = man_dno2.dno.astype(int)
NIH.loc[:, 'dno'] = NIH.dno.astype(int)

time: 17.5 ms


In [56]:
print man_dno2.shape
man_dno_merge = pd.merge(left=man_dno2, right=NIH, on='dno', how='inner')
print man_dno_merge.shape

(103, 86)
(100, 119)
time: 26.3 ms


In [57]:
man_dno_merge2 = man_dno_merge.rename(columns={'clean_first_name_x': 'clean_first_name2', 
                                               'clean_middle_name_x': 'clean_middle_name2',
                             'clean_last_name_x': 'clean_last_name2'})


time: 13.8 ms


In [58]:
fails = man_dno2.loc[~man_dno2.dno.isin(man_dno_merge.dno), :]
print fails.shape

(3, 86)
time: 14.1 ms


In [59]:
# append matches together
match6 = pd.concat([nm_match3, match5, man_dno_merge2], axis=0)

time: 45.9 ms


In [60]:
# get nonmatched NIH people and not matched applicants 
nm_apps2 = get_nonmatched(apps5, id_colname=RAW_CARD_ID, matched_ids=match6[RAW_CARD_ID].dropna().values)
nm_NIH2 = get_nonmatched(NIH, id_colname=NIH_ID, matched_ids=match6[NIH_ID].dropna().values)

(1358, 86)
(1241, 34)
time: 19.5 ms


In [61]:
nm_NIH3 = nm_NIH2.loc[
    (nm_NIH2.eod_year< 1976) & (
        nm_NIH2.eod_year>1963), NAME_COLS+['medical_school', 'eod_year']].sort_values('clean_last_name')
print nm_NIH3.shape

(28, 5)
time: 16.3 ms


In [62]:
apps_match = nm_apps2.loc[nm_apps2.clean_last_name.isin(nm_NIH3.clean_last_name.values)]
test_merge = pd.merge(left=nm_NIH3, right=apps_match, on='clean_last_name', how='inner').sort_values('clean_last_name')
test_merge = test_merge[sorted(test_merge.columns)]
if OUTPUT_CSV:
    test_merge.to_csv(os.path.join(CORRECTIONS_DIR, 'test_merge_missing_NIH.csv'), index=False)

time: 25.6 ms


In [63]:
# test_merge[['clean_first_name_x', 'clean_first_name_y', 'clean_middle_name_x', 'clean_middle_name_y']]

time: 10.6 ms


In [64]:
match6.loc[pd.isnull(match6.clean_last_name), 'clean_last_name'] =  match6.loc[
    pd.isnull(match6.clean_last_name), 'clean_last_name_x']
to_drop = [c for c in match6.columns if c.endswith('_sim') or '_counts' in c]
match7= match6.drop(to_drop+[
        'dup_flag', 'eod_year_diff', 'fuzzy_merge_col_x', 'fuzzy_merge_col_y', 'unknown',
        'Unnamed: 0', 'raw_uuid_duplicate', 'dno_duplicate', 'count_missing',
                'clean_last_name_x', 'clean_last_name_y', 'is_match'], axis=1)

time: 25.2 ms


In [65]:
# consolidate columns in match6
match7a = consolidate_merge_cols(match7, ['_x', '_y'], [])

['citizenship', 'clean_first_initial', 'clean_first_name', 'clean_middle_initial', 'clean_middle_name', 'clean_suffix', 'intern_dates', 'internship_hospital', 'is_female', 'medical_school', 'res_dates', 'residency_hospital', 'residency']
time: 13.9 s


In [66]:
mask = ~pd.isnull(match7a.clean_first_name2)
match7a.loc[mask, 'clean_first_name'] = match7a.loc[mask, 'clean_first_name2']

mask = ~pd.isnull(match7a.clean_last_name2)
match7a.loc[mask, 'clean_last_name'] = match7a.loc[mask, 'clean_last_name2']

mask = ~pd.isnull(match7a.clean_middle_name2)
match7a.loc[mask, 'clean_middle_name'] = match7a.loc[mask, 'clean_middle_name2']

time: 19.7 ms


In [67]:
match8 = pd.concat([
        man_dno[~pd.isnull(man_dno['dno'])], match7a, apps4.loc[
            ~apps4[PERSON_ID].isin(match7a[PERSON_ID].values),:]], axis=0).drop(['clean_first_name2', 
                                                                                'clean_middle_name2', 
                                                                                'clean_last_name2'], axis=1)
print sorted(match8.columns)
print match8.shape

['address', 'age', 'application_date', 'application_year', 'application_year_1', 'application_year_2', 'application_year_3', 'associate_program_entered', 'bob', 'ca', 'cc', 'citizenship', 'city', 'clean_college_trans', 'clean_first_initial', 'clean_first_name', 'clean_first_name_y', 'clean_last_name', 'clean_last_name_y', 'clean_middle_initial', 'clean_middle_name', 'clean_middle_name_y', 'clean_suffix', 'clinical', 'cord', 'data_source', 'date_of_birth', 'dbs', u'dno', 'dob', 'duplicate_dno', 'eod_year', 'fifth', 'first_name', 'fuzzy_merge_col', 'generation', 'honor_societies_first', 'honor_societies_fourth', 'honor_societies_second', 'honor_societies_third', 'ic', 'institute', 'intern_dates', 'internship_hospital', 'internship_year(s)', 'is_female', 'lab_brch', 'last_name', 'med_school', 'medical_school', 'medschool_year_grad', 'middle_name', 'nci', 'nei', 'nhi', 'nhli', 'niaid', 'niamd', 'niamdd', 'nichd', 'nichhd', 'nidr', 'niehs', 'nigms', 'nimh', 'nindb', 'ninds', 'oir', 'origina

In [68]:
# replace date of birth with dob whenever date of birth missing and dob is not
match8.loc[
    (pd.isnull(match8['date_of_birth'])) & (~pd.isnull(match8['dob'])), 'date_of_birth'] = match8.loc[
        (pd.isnull(match8['date_of_birth'])) & (~pd.isnull(match8['dob'])), 'dob']


c1 = 'date_of_birth'
c2 = 'dob'
match8.loc[(pd.isnull(match8[c1])) & (~pd.isnull(match8[c2])), [c1, c2]]

,date_of_birth,dob


time: 23.1 ms


In [69]:
match8.loc[(match8.clean_last_name=='KNOWLER') & (match8.clean_first_name=='JAN'), 'dno'] = 1922
match8.loc[
    (match8.clean_last_name== 'E ROSS HARVARD') & (match8.clean_first_name=='MICHAEL'), 'clean_last_name'] = 'ROSS'


time: 16.8 ms


In [70]:
match9 = match8.drop(['dob'], axis=1).reset_index(drop=True).sort_values(
    NAME_COLS+['dno']).drop_duplicates(NAME_COLS+['dno']).drop_duplicates(NAME_COLS+[PERSON_ID])
# print sorted(match9.columns)
print match8.shape
print match9.shape

(4270, 103)
(4159, 102)
time: 44 ms


In [71]:
# match9.loc[(~pd.isnull(match9.clean_first_name_y)), ['clean_first_name', 'clean_first_name_y']]

time: 14.9 ms


In [72]:
match9.is_female.describe()

/home/lraymond/MIT/Azoulay_2016/yellow_berets/yb/local/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


count    4153.000000
mean        0.016614
std         0.127837
min         0.000000
25%              NaN
50%              NaN
75%              NaN
max         1.000000
Name: is_female, dtype: float64

time: 60.2 ms


In [73]:
# add an international applicant flag
# also drop anyone missing first, middle names and med school
match9['is_foreign'] = 0
match9.loc[:, 'is_foreign'] = match9.medical_school.apply(is_foreign_med_school)

time: 2.57 s


In [74]:
match9.is_foreign.describe()

count    4158.000000
mean        0.002165
std         0.046479
min         0.000000
25%              NaN
50%              NaN
75%              NaN
max         1.000000
Name: is_foreign, dtype: float64

time: 14.9 ms


In [75]:
match9['control_flag'] = 0
match9.loc[pd.isnull(match9.dno) & pd.isnull(match9.year_accepted), 'control_flag'] = 1

time: 17.9 ms


In [76]:
match9.loc[pd.isnull(match9.application_year), 'application_year'] = match9.loc[
    pd.isnull(match9.application_year), 'application_year_1']

time: 13.3 ms


In [77]:
def total_number_applications(app_years):
    unique_apps_years = app_years.dropna().unique()
    return unique_apps_years.shape[0]

match9['number_applications'] = match9[
    ['application_year', 'application_year_1', 'application_year_2', 'application_year_3']].apply(
        total_number_applications, axis=1)

time: 1.53 s


In [78]:
def applications_max_min(app_years, fnc):
    unique_apps_years = app_years.dropna().unique()
    if unique_apps_years.shape[0] == 0:
        print app_years
        return np.nan
    return fnc(unique_apps_years)

time: 11.4 ms


In [79]:
match9['application_year_max'] = match9[
    ['application_year', 'application_year_1', 'application_year_2', 'application_year_3']].apply(
        funcy.rpartial(applications_max_min, max), axis=1)


application_year     NaN
application_year_1   NaN
application_year_2   NaN
application_year_3   NaN
Name: 3962, dtype: float64
application_year     NaN
application_year_1   NaN
application_year_2   NaN
application_year_3   NaN
Name: 3730, dtype: float64
application_year     NaN
application_year_1   NaN
application_year_2   NaN
application_year_3   NaN
Name: 3731, dtype: float64
time: 1.54 s


In [80]:
match9['application_year_min'] = match9[
    ['application_year', 'application_year_1', 'application_year_2', 'application_year_3']].apply(
        funcy.rpartial(applications_max_min, min), axis=1)


application_year     NaN
application_year_1   NaN
application_year_2   NaN
application_year_3   NaN
Name: 3962, dtype: float64
application_year     NaN
application_year_1   NaN
application_year_2   NaN
application_year_3   NaN
Name: 3730, dtype: float64
application_year     NaN
application_year_1   NaN
application_year_2   NaN
application_year_3   NaN
Name: 3731, dtype: float64
time: 1.56 s


In [81]:
match9['time_period_flag'] = 0
match9.loc[(match9.application_year_max>1964) & (match9.application_year_max<1976), 'time_period_flag'] = 1

time: 17.7 ms


In [82]:
# delete people missing first and middle names
print match9.shape
match9A = match9.loc[(~pd.isnull(match9.clean_first_name) & ~pd.isnull(match9.clean_middle_name)), :]
print match9A.shape

(4159, 108)
(3617, 108)
time: 17.3 ms


In [83]:
match9A.rename(columns={'res_dates': 'residency_dates', 'intern_dates': 'internship_dates', 
                      'clean_college_trans': 'clean_college'}, inplace=True)

IMPORTANT_COLS = [NIH_ID, PERSON_ID, 'application_year_min', 'application_year_max', 'eod_year',
                  'clean_first_name', 'clean_middle_name', 
                 'clean_last_name', 'control_flag', 'time_period_flag', 'year_accepted', 'rejected', 'rejection_date', 'clean_college', 'medical_school',
                'residency_dates', 'internship_dates', 'is_female', 'is_foreign', 'number_applications']

other_cols = sorted([i for i in match9A.columns if i not in IMPORTANT_COLS])

# order columns so important ones are 
match10 = match9A[IMPORTANT_COLS+other_cols].sort_values(['clean_last_name', 'clean_first_name', 'application_year_max']).drop(
    ['clean_first_name_y', 'clean_middle_name_y', 'clean_last_name_y'], axis=1)

match11 = match10.dropna(subset=[PERSON_ID], axis=0).sort_values(['clean_last_name', 'clean_first_name'])

# wide_apps5.to_pickle(os.path.join(APP_DATA_DIR, 'all_apps_plus_NIH_info.p'))
match11.to_csv(os.path.join(APP_DATA_DIR, 'fuzzy_all_apps_plus_NIH_info.csv'), index=False)

/home/lraymond/MIT/Azoulay_2016/yellow_berets/yb/local/lib/python2.7/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


time: 348 ms


In [84]:
ln = 'RING'
match11.loc[match11.clean_last_name=='MILLER', NAME_COLS+[PERSON_ID, 'medical_school', 'application_year', 'dno', 'residency','residency_hospital',
                                                            'internship_hospital', 'residency_dates']]

,clean_middle_name,clean_last_name,clean_first_name,person_uuid,medical_school,application_year,dno,residency,residency_hospital,internship_hospital,residency_dates
1182,EDGAR,MILLER,A,0.0,UNIVERSITY OF MIAMI SCHOOL OF MEDICINE,1969.0,2539.0,DERMATOLOGY,UNIVERSITY OF MIAMI,UNIVERSITY OF MIAMI,1970-1971
3295,MICHAEL,MILLER,ALAN,54.0,ALBANY MEDICAL COLLEGE,1970.0,NaN,NaN,NaN,MOUNT SINAI MEDICAL CENTER,NaN
1181,LEWIS,MILLER,ALEXANDER,91.0,WASHINGTON UNIVERSITY SCHOOL OF MEDICINE,1970.0,2540.0,Dermatology,NaN,JEWISH HOSPITAL OF SAINT LOUIS,1970-1971
1180,PHILIP,MILLER,DAVID,703.0,UNIVERSITY OF PITTSBURGH SCHOOL OF MEDICINE,1969.0,2541.0,MEDICINE,GEORGETOWN UNIVERSITY,GEORGETOWN UNIVERSITY,1970
20,MAX,MILLER,DONALD,823.0,DUKE MEDICAL SCHOOL,1973.0,2542.0,MEDICINE,PETER BENT BRIGHAM HOSPITAL,PETER BENT BRIGHAM HOSPITAL,1974-1975
1179,J,MILLER,EDWARD,899.0,NaN,1966.0,2543.0,NaN,NaN,NaN,NaN
2984,TOBIAS,MILLER,FRANK,1046.0,YALE UNIVERSITY SCHOOL OF MEDICINE,1971.0,NaN,NaN,NaN,MOUNT ZION HOSPITAL AND MEDICAL CENTER,NaN
1177,F,MILLER,GEORGE,1147.0,GEORGE WASHINGTON UNIVERSITY SCHOOL OF MEDICINE,1958.0,2546.0,NaN,NaN,Duke University Hospital,NaN
83,BAYLESS,MILLER,HALBERT,1269.0,UNIVERSITY OF TENNESSEE HEALTH SCIENCE CENTER ...,1967.0,2548.0,PSYCHIATRY,MASSACHUSETTS MENTAL HEALTH CENTER,STANFORD UNIVERSITY HOSPITAL,1968-1969
1176,ISAAC,MILLER,HENRY,1349.0,UCSD SCHOOL OF MEDICINE,1974.0,2549.0,MEDICINE,BETH ISRAEL DEACONESS MEDICAL CENTER,BETH ISRAEL DEACONESS MEDICAL CENTER,1976-1977


time: 53.5 ms


In [85]:
wide_apps_v.loc[wide_apps_v.clean_last_name=='KRUGMAN', NAME_COLS+['dno', PERSON_ID]]

NameError: name 'wide_apps_v' is not defined

time: 97.5 ms


In [ ]:
wide_apps_v.loc[pd.isnull(wide_apps_v.clean_first_name), NAME_COLS+['dno', 'medical_school', 'application_year']].to_csv('extras.csv')

In [ ]:
apps5.loc[apps5.clean_last_name=='PRZYBYLA', NAME_COLS+['medical_school', PERSON_ID]]